Bibliotecas usadas

In [1]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os

Funções auxiliares

In [2]:
def get_url_infos(url): 
    # Faz uma solicitação HTTP para a URL
    response = requests.get(url)

    # Verifica se a solicitação foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Analisa o conteúdo da página usando BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        try:
            assunto = soup.find_all('span', class_='titulo')[0].text
        except:
            assunto = None
        try:
            titulo = soup.find_all('h1', class_='entry-title')[0].text
        except:
            titulo = None
        try:
            subtitulo = soup.find_all('h2')[0].text
        except:
            subtitulo = None
        try:
            local = soup.find_all('span', class_= "locality")[0].text
        except:
            local = None
        try:
            artigo = ''
            n_paragrafos = 0
            for i in soup.find_all('p')[2:-4]:
                artigo = artigo + i.text
                n_paragrafos += 1
        except:
            artigo = None
            n_paragrafos = None
        return assunto, titulo, subtitulo, local, artigo, n_paragrafos
    return None, None, None, None, None, None

Importação do dataframe

In [4]:
df = pd.read_csv('data\Historico_de_materias.csv')

# criando colunas para extração
df['assunto_ex']   = None
df['titulo_ex']    = None
df['subtitulo_ex'] = None
df['local_ex']     = None
df['artigo_ex']    = None
df['n_paragrafos'] = None
df.head()

,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto,assunto_ex,titulo_ex,subtitulo_ex,local_ex,artigo_ex,n_paragrafos
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia,None,None,None,None,None,None
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia,None,None,None,None,None,None
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia,None,None,None,None,None,None
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia,None,None,None,None,None,None
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia,None,None,None,None,None,None


Web scrapping

In [29]:
##########################################################################################################################
#                                           CUIDADO AO RODAR                                                             #
##########################################################################################################################

#Nessa celula dividimos o dataframe em varios para organizar a extração e salvar em partes

n_linhas = 50 # numero de linhas de cada sub dataframe 
diretorio = 'data/to_enrich'

certeza = input('TEM CERTEZA QUE QUER RODAR: ')

if certeza == 'CERTEZA':

    #limpando o diretorio
    for root, dirs, files in os.walk(diretorio):
        for arquivo in files:
            caminho_completo = os.path.join(root, arquivo)
            if os.path.exists(caminho_completo):
                os.remove(caminho_completo)

    #criando os arquivos para enriquecer 
    for i in range(0, df.shape[0], n_linhas):
        dfi = df.iloc[i:i+n_linhas]
        dfi.to_csv(f'data/to_enrich/df_i{i:05}_i{i+n_linhas:05}.csv', index = False)

In [6]:
diretorio = 'data/to_enrich'


#percorre cada arquivo do diretorio
for root, dirs, files in os.walk(diretorio):
    for arquivo in files:
        caminho_completo = os.path.join(root, arquivo)
        print(caminho_completo)
        dfi = pd.read_csv(caminho_completo)

        #percorre o subdataframe 
        for i in range(dfi.shape[0]): 
            url = dfi['url_noticia'].loc[i]

            #aplica a função de extrair informações da url e coloca no df
            assunto, titulo, subtitulo, local, artigo, n_paragrafos = get_url_infos(url)
            dfi['assunto_ex'].iloc[i]   = assunto
            dfi['titulo_ex'].iloc[i]    = titulo
            dfi['subtitulo_ex'].iloc[i] = subtitulo
            dfi['local_ex'].iloc[i]     = local
            dfi['artigo_ex'].iloc[i]    = artigo
            dfi['n_paragrafos'].iloc[i] = n_paragrafos

            # apenas para informar o progresso
            if (i/dfi.shape[0])*100 % 10 == 0:
                print(f'{(i/dfi.shape[0])*100}%')

        # salva o df no diretorio enriquecido
        dfi.to_csv(caminho_completo.replace('to_', ''))

        # apaga o arrquivo ja enriquecido 
        if os.path.exists(caminho_completo):
            os.remove(caminho_completo)



data/to_enrich\df_i04100_i04150.csv


c:\Users\55119\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04150_i04200.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04200_i04250.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04250_i04300.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04300_i04350.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04350_i04400.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04400_i04450.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04450_i04500.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04500_i04550.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04550_i04600.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
data/to_enrich\df_i04600_i04650.csv
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.

ConnectionError: ('Connection aborted.', OSError("(10053, 'WSAECONNABORTED')"))

In [5]:
# concatena as os df enriquecidos e salva em um unico

diretorio = 'data/enrich'

df[df['assunto'] == 'triangulo'].to_csv('data/materias_enriquecida.csv', index = False)

for root, dirs, files in os.walk(diretorio):
    for arquivo in files:

        caminho_completo = os.path.join(root, arquivo)
        print(caminho_completo)

        df  = pd.read_csv('data/materias_enriquecida.csv')
        dfi = pd.read_csv(caminho_completo).drop(columns = 'Unnamed: 0')

        df = pd.concat([df, dfi], ignore_index=True)
        df.to_csv('data/materias_enriquecida.csv', index = False)

data/enrich\df_i00000_i00050.csv
data/enrich\df_i00050_i00100.csv
data/enrich\df_i00100_i00150.csv
data/enrich\df_i00150_i00200.csv
data/enrich\df_i00200_i00250.csv
data/enrich\df_i00250_i00300.csv
data/enrich\df_i00300_i00350.csv
data/enrich\df_i00350_i00400.csv
data/enrich\df_i00400_i00450.csv
data/enrich\df_i00450_i00500.csv
data/enrich\df_i00500_i00550.csv
data/enrich\df_i00550_i00600.csv
data/enrich\df_i00600_i00650.csv
data/enrich\df_i00650_i00700.csv
data/enrich\df_i00700_i00750.csv
data/enrich\df_i00750_i00800.csv
data/enrich\df_i00800_i00850.csv
data/enrich\df_i00850_i00900.csv
data/enrich\df_i00900_i00950.csv
data/enrich\df_i00950_i01000.csv
data/enrich\df_i01000_i01050.csv
data/enrich\df_i01050_i01100.csv
data/enrich\df_i01100_i01150.csv
data/enrich\df_i01150_i01200.csv
data/enrich\df_i01200_i01250.csv
data/enrich\df_i01250_i01300.csv
data/enrich\df_i01300_i01350.csv
data/enrich\df_i01350_i01400.csv
data/enrich\df_i01400_i01450.csv
data/enrich\df_i01450_i01500.csv
data/enric

In [32]:
pd.read_csv('data/materias_enriquecida.csv')

,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto,assunto_ex,titulo_ex,subtitulo_ex,local_ex,artigo_ex,n_paragrafos
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia,Economia,"Mesmo com alta do dólar, gastos de brasileiros...","Em 2013, despesas somaram US$ 25 bilhões, 14% ...","Do G1, em Brasília","\n\tA alta de 15% no dólar em 2013, a maior do...",13.0
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia,Economia,"Para Dilma, é 'apressada' a tese de que emerge...",Presidente discursou no Fórum Econômico Mundia...,"Do G1, em Brasília",\n\tA presidente Dilma Rousseff afirmou nesta ...,18.0
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia,Economia,"'Temos sido capazes de reduzir a inflação', di...",Presidente do BC participa do Fórum Econômico ...,"Do G1, em São Paulo","\n\tO presidente do Banco Central, Alexandre T...",12.0
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia,veja também,Governo argentino autoriza compra de dólares a...,"Na quinta-feira, taxa de câmbio teve maior que...","Do G1, em São Paulo",\n Nordeste\n ...,22.0
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia,Economia,Petrobras desiste de parte da área de Bem-te-v...,Estatal confirmou 'grande potencial' em Carcar...,"Do G1, em São Paulo",\n\tA Petrobras afirmou que propôs à Agência N...,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1545,2015-05-23,https://web.archive.org/web/20150526174527/htt...,http://globoesporte.globo.com/basquete/noticia...,Cotovelada pode tirar Howard do 5º jogo da fin...,Apesar da boa vitória no jogo 4 da final da Co...,esportes,NaN,Cotovelada pode tirar Howard do 5º jogo da fin...,Incidente aconteceu no 3º período da quarta pa...,"Houston, EUA",Apesar da boa vitória no jogo 4 da final da Co...,22.0
1546,2015-05-23,https://web.archive.org/web/20150526174527/htt...,http://sportv.globo.com/site/combate/noticia/2...,"Fã de ""Coração Valente"", Thales Leites se empo...","""Devagarzinho."" É assim que Thales Leites espe...",esportes,NaN,"Fã de ""Coração Valente"", Thales Leites se empo...",Atleta enfrenta Michael Bisping dia 18 de julh...,Rio de Janeiro,"""Devagarzinho."" É assim que Thales Leites espe...",161.0
1547,2015-05-23,https://web.archive.org/web/20150526174527/htt...,http://sportv.globo.com/site/programas/sensei-...,"Jacaré se diz pronto para Weidman: ""Meu retros...","Dono de cinco vitórias seguidas no UFC, Ronald...",esportes,NaN,"Jacaré se diz pronto para Weidman: ""Meu retros...",Brasileiro considera justo ser o próximo na li...,Rio de Janeiro,"Dono de cinco vitórias seguidas no UFC, Ronald...",359.0
1548,2015-05-23,https://web.archive.org/web/20150526174527/htt...,http://globoesporte.globo.com/basquete/nbb/not...,"Líder do Circuito Mundial de Surfe, Mineirinho...",O time de basquete do Flamengo recebeu a ilust...,esportes,NaN,"Líder do Circuito Mundial de Surfe, Mineirinho...","Na véspera do embarque para as Ilhas Fiji, tor...",Rio de Janeiro,O time de basquete do Flamengo recebeu a ilust...,11.0
